# Test Flakiness Analysis Notebook

This notebook demonstrates the process of loading test execution data, identifying flaky tests based on execution time variability, and visualizing the distribution of execution times.


# Instructions for Use

- Open your Jupyter Notebook environment: Navigate to the notebooks directory within your project.

- Create a new notebook: If you haven't already, name it flakiness_analysis.ipynb.
Add content: Copy and paste each cell's content into your notebook, matching the cell types (Markdown for descriptive text, Code for Python code).

- Adjust paths and parameters: Ensure the data_path variable in the data loading cell points to your actual dataset within the data folder.

- Execute the notebook: Run each cell in order to perform the analysis and generate visualizations.

- This setup provides a holistic view of how to perform and visualize flakiness analysis within a Jupyter Notebook, offering a practical example that aligns with your project's goals.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load your test data
data_path = '../data/performance_test_results.csv'  # Update this path to your actual data file location
df = pd.read_csv(data_path)

# Display the first few rows to confirm proper loading
df.head()

In [ ]:
# Add a simple heuristic for flakiness: High variability in execution times
# Calculate the standard deviation of execution times for each test
std_devs = df.groupby('TestName')['ExecutionTime'].std().reset_index()
std_devs.rename(columns={'ExecutionTime': 'StdDev'}, inplace=True)

# Merge standard deviations back into the original dataframe
df = df.merge(std_devs, on='TestName')

# Classify tests as 'flaky' based on standard deviation threshold
flakiness_threshold = 0.5  # Define a threshold for flakiness
df['Flaky'] = df['StdDev'].apply(lambda x: 'Yes' if x > flakiness_threshold else 'No')

df.head()

In [ ]:
# Visualize the execution times of tests, highlighting flaky tests
flaky_tests = df[df['Flaky'] == 'Yes']
stable_tests = df[df['Flaky'] == 'No']

plt.figure(figsize=(10, 6))
plt.hist(stable_tests['ExecutionTime'], bins=20, alpha=0.5, label='Stable Tests')
plt.hist(flaky_tests['ExecutionTime'], bins=20, alpha=0.5, color='red', label='Flaky Tests')
plt.xlabel('Execution Time (s)')
plt.ylabel('Number of Tests')
plt.title('Distribution of Test Execution Times')
plt.legend()
plt.show()

In [ ]:
# Analyze the proportion of flaky tests
flakiness_ratio = df['Flaky'].value_counts(normalize=True) * 100
print("Flakiness Ratio:\n", flakiness_ratio)

# Optionally, create a pie chart to visualize the ratio of flaky to stable tests
plt.figure(figsize=(6, 6))
plt.pie(flakiness_ratio, labels=flakiness_ratio.index, autopct='%1.1f%%', startangle=140, colors=['green', 'red'])
plt.title('Flaky vs. Stable Tests Ratio')
plt.show()